In [1]:
import pandas as pd
import numpy as np

print("--- 開始執行資料診斷腳本 ---")

# --- 載入資料 ---
file_path = u'2023資料剖析後內容1~8月芝山到天母0609整理二版.csv'
try:
    df = pd.read_csv(file_path, encoding='utf-8-sig')
except UnicodeDecodeError:
    df = pd.read_csv(file_path, encoding='big5')
except FileNotFoundError:
    print(f"錯誤：找不到檔案 '{file_path}'。請確認檔案名稱和路徑是否正確。")
    exit()

# 移除空行
df.dropna(subset=['Date'], inplace=True)
print(f"原始資料共 {len(df)} 筆。")

# --- 診斷 1：檢查 'Time2' (騎乘時間) 的轉換問題 ---
print("\n--- 診斷 1：檢查 'Time2' 轉換問題 ---")
def check_time2(time_str):
    try:
        pd.to_timedelta(str(time_str).strip())
        return True # 格式正確
    except:
        return False # 格式錯誤

# 找出所有格式錯誤的 'Time2' 值
problematic_time2 = df[~df['Time2'].apply(check_time2)]

if not problematic_time2.empty:
    print(f"發現 {len(problematic_time2)} 筆 'Time2' 格式有問題。")
    print("有問題的 'Time2' 值的範例（最多顯示前 20 個不重複的值）：")
    print(problematic_time2['Time2'].unique()[:20])
else:
    print("'Time2' 欄位的所有值格式看起來都可被解析。")

# --- 診斷 2：檢查 'Date' 和 'Time1' (出發時間) 的轉換問題 ---
print("\n--- 診斷 2：檢查 'Date' 與 'Time1' 轉換問題 ---")
# 先確保 Date 和 Time1 是乾淨的字串
df['Date_str'] = df['Date'].astype(int).astype(str)
df['Time1_str'] = df['Time1'].astype(str).str.strip()

# 使用 errors='coerce'，如果格式錯誤，會直接變成 NaT (Not a Time)
df['Datetime_check'] = pd.to_datetime(df['Date_str'] + ' ' + df['Time1_str'], format='%Y%m%d %H:%M:%S', errors='coerce')

# 找出所有日期時間轉換失敗的行
problematic_datetime = df[df['Datetime_check'].isna()]

if not problematic_datetime.empty:
    print(f"發現 {len(problematic_datetime)} 筆 'Date'/'Time1' 組合格式有問題。")
    print("有問題的 'Date'/'Time1' 組合範例（最多顯示前 10 筆）：")
    print(problematic_datetime[['Date', 'Time1']].head(10))
else:
    print("'Date' 和 'Time1' 欄位的組合格式看起來都可被解析。")
    
# --- 診斷 3：檢查篩選條件 ---
print("\n--- 診斷 3：檢查篩選條件 ---")
# 應用轉換（錯誤返回 NaN）
def parse_time_to_seconds(time_str):
    try:
        return pd.to_timedelta(str(time_str).strip()).total_seconds()
    except:
        return np.nan

df['TotalSeconds_check'] = df['Time2'].apply(parse_time_to_seconds)

# 分別計算每個條件會篩選掉多少資料
total_rows = len(df)
time2_valid = df['TotalSeconds_check'].notna()
datetime_valid = df['Datetime_check'].notna()
seconds_in_range = (df['TotalSeconds_check'] >= 180) & (df['TotalSeconds_check'] <= 3600)

print(f"總筆數: {total_rows}")
print(f"滿足 'Time2' 可解析的筆數: {time2_valid.sum()}")
print(f"滿足 'Datetime' 可解析的筆數: {datetime_valid.sum()}")
print(f"滿足 'TotalSeconds' 在 3-60 分鐘內的筆數: {seconds_in_range.sum()} (在可解析的基礎上)")

# 計算同時滿足所有條件的筆數
final_count = (time2_valid & datetime_valid & seconds_in_range).sum()
print(f"\n>> 同時滿足所有條件的最終筆數: {final_count}")

print("\n--- 診斷結束 ---")

--- 開始執行資料診斷腳本 ---
原始資料共 5016 筆。

--- 診斷 1：檢查 'Time2' 轉換問題 ---
發現 5015 筆 'Time2' 格式有問題。
有問題的 'Time2' 值的範例（最多顯示前 20 個不重複的值）：
['16:07' '21:22' '09:39' '12:06' '10:59' '11:13' '12:54' '08:54' '10:19'
 '11:10' '13:54' '22:28' '22:54' '22:53' '11:36' '10:23' '15:13' '16:36'
 '12:31' '11:53']

--- 診斷 2：檢查 'Date' 與 'Time1' 轉換問題 ---
'Date' 和 'Time1' 欄位的組合格式看起來都可被解析。

--- 診斷 3：檢查篩選條件 ---
總筆數: 5016
滿足 'Time2' 可解析的筆數: 1
滿足 'Datetime' 可解析的筆數: 5016
滿足 'TotalSeconds' 在 3-60 分鐘內的筆數: 0 (在可解析的基礎上)

>> 同時滿足所有條件的最終筆數: 0

--- 診斷結束 ---
